In [1]:
import pyspark as SparkContext
import warnings
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import desc

try:
    sc.stop()
except:
    pass

try:
    # create SparkContext on all CPUs available: in my case I have 4 CPUs on my laptop
    sc = SparkContext.SparkContext('local[4]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    warnings.warn("SparkContext already exists in this scope")
    
ssc = StreamingContext(sc, 5 )
sqlContext = SQLContext(sc)
ssc.checkpoint( "/Users/anujchaudhari/Desktop/256/project/samples/twitter_streaming/checkpoint")

socket_stream = ssc.socketTextStream("192.168.0.100", 5556)

lines = socket_stream.window( 5 )

from nltk.tokenize import WordPunctTokenizer
import re
from bs4 import BeautifulSoup
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')


Just created a SparkContext


In [2]:
import time

def tweet_cleaner_updated(text):
    print(text)
    return text

# Lazily instantiated global instance of SparkSession
def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]


def process(time, rdd):
    print("========= %s =========" % str(time))
    try:
        # Get the singleton instance of SparkSession
        spark = getSparkSessionInstance(rdd.context.getConf())

        # Convert RDD[String] to RDD[Row] to DataFrame
        rowRdd = rdd.map(lambda w: Row(word=w))
        wordsDataFrame = spark.createDataFrame(rowRdd)

        # Creates a temporary view using the DataFrame
#         wordsDataFrame.createOrReplaceTempView("text")

#         # Do word count on table using SQL and print it
#         wordCountsDataFrame = spark.sql("select * from text ")
#         wordCountsDataFrame.show()
    except:
        print(" exception ")
        pass

lines = lines.map(lambda x: x.lower());
lines = lines.map(lambda x: x.replace(" rt " , " "))
lines = lines.map(lambda x: x.replace("\n" , " "))
lines = lines.reduce(lambda x,y : x + y)
lines = lines.map(lambda x: x.split(" $$$$$$ "))
# lines = lines.map(lambda x: tweet_cleaner_updated(x))

#df = sqlContext.createDataFrame(lines.collect(), ["text"])

#lines.foreachRDD(process)

#lines.saveAsTextFiles("x.csv")
lines.pprint()

ssc.start()


-------------------------------------------
Time: 2018-04-22 13:50:05
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 13:50:10
-------------------------------------------
['1 :  @kp24: sky’s commentators now cashing in on the @ipl. the same ones that played a major part in ending my england career. ']

-------------------------------------------
Time: 2018-04-22 13:50:15
-------------------------------------------

-------------------------------------------
Time: 2018-04-22 13:50:20
-------------------------------------------



In [ ]:
ssc.stop()